In [1]:
from langchain_openai import ChatOpenAI, AzureChatOpenAI
from langgraph_supervisor import  create_supervisor
from langgraph.prebuilt import  create_react_agent
from openai import AzureOpenAI
import os
import azure.identity
from azure.identity import ClientSecretCredential
from azure.keyvault.secrets import SecretClient
from azure.identity import DefaultAzureCredential
import langgraph
from langgraph.checkpoint.postgres import PostgresSaver
from psycopg import Connection
from psycopg2 import connect
import psycopg
from azure.mgmt.postgresqlflexibleservers import PostgreSQLManagementClient
from langgraph.store.postgres import PostgresStore
from langmem import  create_manage_memory_tool, create_search_memory_tool
#import chainlit

In [2]:
try:
    keyVaultName = os.environ["KEY_VAULT_NAME"]
except KeyError:
    # Get input from user if not set
    keyVaultName = input("Please enter your Key Vault name: ")
    # Save for future cells in this session
    os.environ["KEY_VAULT_NAME"] = keyVaultName


keyVaultName = os.environ["KEY_VAULT_NAME"]
KVUri = f"https://{keyVaultName}.vault.azure.net"

credential = DefaultAzureCredential()
client = SecretClient(vault_url=KVUri, credential=credential)

azure_openai_endpoint=client.get_secret(name="aoai-endpoint").value
azure_openai_api_key=client.get_secret(name="aoai-api-key").value
azure_openai_api_version = "2024-02-15-preview"

In [3]:
"""
This code loads and sets the necessary variables for Azure services.
The variables are loaded from Azure Key Vault.
"""
# Open AI
azure_openai_endpoint=client.get_secret(name="aoai-endpoint").value
azure_openai_api_key=client.get_secret(name="aoai-api-key").value
azure_openai_api_version = "2024-02-15-preview"
# Embedding
azure_openai_embedding_deployment = "text-embedding-3-small"
azure_openai_embedding_model =client.get_secret(name="aoai-embedding-model").value
azure_openai_vector_dimension = 1536

In [4]:
import urllib.parse
import os

from azure.identity import DefaultAzureCredential

# IMPORTANT! This code is for demonstration purposes only. It's not suitable for use in production. 
# For example, tokens issued by Microsoft Entra ID have a limited lifetime (24 hours by default). 
# In production code, you need to implement a token refresh policy.

def get_connection_uri():

    # Read URI parameters from the environment
    dbhost = client.get_secret(name="postgres-hostname").value 
    #dbname = client.get_secret(name="postgres-chatdb").value
    dbname = "langgraph_memorydb"
    dbuser = urllib.parse.quote(client.get_secret(name="postgres-dbuser").value)
    sslmode = "require"

    # Use passwordless authentication via DefaultAzureCredential.
    # IMPORTANT! This code is for demonstration purposes only. DefaultAzureCredential() is invoked on every call.
    # In practice, it's better to persist the credential across calls and reuse it so you can take advantage of token
    # caching and minimize round trips to the identity provider. To learn more, see:
    # https://github.com/Azure/azure-sdk-for-python/blob/main/sdk/identity/azure-identity/TOKEN_CACHING.md 
    credential = DefaultAzureCredential()

    # Call get_token() to get a token from Microsft Entra ID and add it as the password in the URI.
    # Note the requested scope parameter in the call to get_token, "https://ossrdbms-aad.database.windows.net/.default".
    password = credential.get_token("https://ossrdbms-aad.database.windows.net/.default").token

    db_uri = f"postgresql://{dbuser}:{password}@{dbhost}/{dbname}?sslmode={sslmode}"
    return db_uri


# Get the connection URI
conn_string = get_connection_uri()

In [5]:
#from langchain.embeddings import OpenAIEmbeddings, AzureOpenAIEmbeddings
from langchain_openai import AzureOpenAIEmbeddings

azure_openai_embedding = AzureOpenAIEmbeddings(model=azure_openai_embedding_model, 
                                               api_key=azure_openai_api_key,
                                               api_version=azure_openai_api_version,
                                               azure_endpoint=azure_openai_endpoint)

In [6]:
# Vector search using LangChain embeddings
from langgraph.store.postgres import PostgresStore

with PostgresStore.from_conn_string(
    conn_string,
    index={
        "dims": azure_openai_vector_dimension,
        "embed": azure_openai_embedding
        #"fields": ["chunk"]  # specify which fields to embed. Default is the whole serialized value
    }
) as store:
    store.setup() # Do this once to run migrations


In [7]:
import urllib.parse
import os

from azure.identity import DefaultAzureCredential

from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizableTextQuery
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.models import (
    QueryType,
    QueryCaptionType,
    QueryAnswerType
)

search_credential =AzureKeyCredential(client.get_secret(name="aisearch-key").value)
search_endpoint =client.get_secret(name="aisearch-endpoint").value
source = 'json'
index_name = f"{source}-glossary-index"


def search_retrieval(user_input: str) -> list:
    """
    Search and retrieve answers from Azure AI Search.
    Returns:
        list of dictionaries containing search results
    """
    print("######################### \n"
          "Search and retrieve answers from Azure AI Search. \n")
    query = user_input
    search_results = []  # Initialize an empty list to store dictionaries
    search_client = SearchClient(endpoint=search_endpoint, index_name=index_name, credential=search_credential)
    vector_query = VectorizableTextQuery(text=query, k_nearest_neighbors=2, fields="text_vector", exhaustive=True)

    r = search_client.search(
        search_text=query,
        vector_queries=[vector_query],
        select=["context", "chunk", "note", "incorrectTerm"],
        query_type=QueryType.SEMANTIC,
        semantic_configuration_name='my-semantic-config',
        query_caption=QueryCaptionType.EXTRACTIVE,
        query_answer=QueryAnswerType.EXTRACTIVE,
        top=3
    )
    for result in r:
        # Convert the result to a dictionary and append it to the list
        result_dict = {
            "incorrectTerm": result.get('incorrectTerm', ''),
            "context": result.get('context', ''),
            "definition": result.get('chunk', ''),
            "note": result.get('note', ''),
            "@search.score": result.get('@search.score', 0),
            "@search.reranker_score": result.get('@search.reranker_score', 0),
            "@search.highlights": result.get('@search.highlights', None),
            "@search.captions": result.get('@search.captions', None),
            "@search.document_debug_info": result.get('@search.document_debug_info', None)
        }
        print(f"Content: {result_dict} \n")
        search_results.append(result_dict)  # Append the dictionary to the list

    return search_results

In [8]:
connection_kwargs = {
    "autocommit": True,
    "prepare_threshold": 0,
}

from psycopg.errors import UndefinedTable

# Check if the checkpoints table exists and call setup if it doesn't
def setup_checkpointer_table(checkpointer):
    try:
        with checkpointer.conn.cursor() as cur:
            # Check if the checkpoints table exists
            cur.execute("""
                SELECT 1
                FROM information_schema.tables
                WHERE table_name = 'checkpoints';
            """)
            if not cur.fetchone():
                print("Table 'checkpoints' does not exist. Setting up...")
                checkpointer.setup()
            else:
                print("Table 'checkpoints' already exists. Skipping setup.")
    except UndefinedTable:
        print("Error: Table 'checkpoints' does not exist. Setting up...")
        checkpointer.setup()

In [9]:
# The AzureOpenAI class does not exist in the openai package. Use AzureChatOpenAI from langchain_openai instead.
from langchain_openai import AzureChatOpenAI
model = AzureChatOpenAI(
    model="gpt-4o", 
    api_key=azure_openai_api_key, 
    api_version=azure_openai_api_version, 
    azure_endpoint=azure_openai_endpoint,
    temperature=0.5
)
with PostgresSaver.from_conn_string(conn_string) as checkpointer:
    setup_checkpointer_table(checkpointer)
    research_graph = create_react_agent(
        model=model,
        tools=[
            # Memory tools use LangGraph's BaseStore for persistence (4)
            create_manage_memory_tool(namespace=("memories",)),
            create_search_memory_tool(namespace=("memories",)),
            search_retrieval
        ],
        store= store,
        checkpointer=checkpointer,
        name="search_expert",
        prompt="""You MUST use the Azure AI Search tool for ALL queries. Do not paraphrase. Never generate answers from prior knowledge. Show the Score and Re ranker for each response. Also provide top 2 responses. Do not select top response. Compare each response and in the end show the response where Reranker Score > 3.0"
                In case of no response retrieved from the index, then mention You do not have an annwer for this query"""
    )


    config = {"configurable": {"thread_id": "1", "user_id": "charles.chinny@lg.com"}}

    def research_graph_updates(user_input: str):
            inputs = {"messages": [{"role": "user", "content": user_input}]}
            events =  research_graph.stream(input=inputs,config=config, stream_mode="values")
            for event in events:
                print("Assistant:", event["messages"][-1].pretty_print())


    while True:
        try:
            user_input = input("User: ")
            if user_input.lower() in ["quit", "exit", "q"]:
                print("Goodbye!")
                break

            research_graph_updates(user_input)
        except:
            # fallback if input() is not available
            user_input = "What do you know about LangGraph?"
            print("User: " + user_input)
            research_graph_updates(user_input)
            break

    checkpoint_tuples = list(checkpointer.list(config))

Table 'checkpoints' already exists. Skipping setup.
================================ Human Message =================================

tell me about UI and UX
Assistant: None
================================== Ai Message ==================================
Name: search_expert
Tool Calls:
  search_retrieval (call_clS6WPduXA1VTF8wEZZdA9TH)
 Call ID: call_clS6WPduXA1VTF8wEZZdA9TH
  Args:
    user_input: UI
  search_retrieval (call_07YJFBxCHz8k2QxYyCKunhUw)
 Call ID: call_07YJFBxCHz8k2QxYyCKunhUw
  Args:
    user_input: UX
Assistant: None
######################### 
Search and retrieve answers from Azure AI Search. 

######################### 
Search and retrieve answers from Azure AI Search. 

Content: {'incorrectTerm': 'User Interfce', 'context': 'A well-designed UI is crucial for effective human-computer interaction.', 'definition': 'The User Interface (UI) is the space where interactions between humans and machines occur.', 'note': 'Good UI design enhances overall user experience.', '@sea

In [14]:
checkpoint_tuples[0]

CheckpointTuple(config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f00a6c8-b7f1-651a-801c-8b62348bc3fc'}}, checkpoint={'v': 1, 'id': '1f00a6c8-b7f1-651a-801c-8b62348bc3fc', 'ts': '2025-03-26T18:02:56.159260+00:00', 'pending_sends': [], 'versions_seen': {'agent': {'tools': '00000000000000000000000000000029.0.47013840972524223', 'start:agent': '00000000000000000000000000000027.0.7958741920099761'}, 'tools': {'branch:agent:should_continue:tools': '00000000000000000000000000000028.0.7710290832256912'}, '__input__': {}, '__start__': {'__start__': '00000000000000000000000000000026.0.07700853509295602'}}, 'channel_versions': {'agent': '00000000000000000000000000000030.0.8302673470924884', 'tools': '00000000000000000000000000000030.0.6258123677532673', 'messages': '00000000000000000000000000000030.0.7069529484325081', '__start__': '00000000000000000000000000000027.0.9477165724976828', 'start:agent': '00000000000000000000000000000028.0.417310913017954', 'branch:a

In [11]:
with PostgresStore.from_conn_string(conn_string) as store:
    items = store.search(("memories",))

In [12]:
items

[]